<h3> Summary of notebook: </h3>

- The notebook "translator_transformer_tidy_mode_v1.ipynb" has all the pieces.
- Here I separate the code into a script for training and a script for translating
- Use this script ("train_v1.ipynb") to train the model; save the source arrays, target arrays, dictionaries, model weights
- Use the next script ("translate_v1.ipynb") to load the model and predict
- *** In "translate_v1.ipynb", you need to instantiate the Transformer with the same parameters from the "train_v1.ipynb" script ***
- Then you load the model weights and use the functions to predict new sentences + plot attention weights

- In google colab, I experimented with some sets of (hyper) parameters, using 10% of the data (see the file "Training_Observations.docx")
- So far, the most promising are: embedding_dim = 256, num_heads = 8, num_layers = 4, fully_connected_dim = 512
- Increasing any of the parameters meant the accuracy reached a peak and then decreased to a useless value. 
- You could investigate further why this happened; with "Params 3", I also tried training with 50% of the data, but the accuracy still showed the strange behaviour.

In [8]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, Concatenate, LSTM, Embedding, Dense, MultiHeadAttention, LayerNormalization, Dropout
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.initializers import Constant
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import re
import os
import io
import time

In [9]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd gdrive/MyDrive/ColabNotebooks/train_translate

# df_en_de = pd.read_table('/content/gdrive/MyDrive/ColabNotebooks/train_translate/df_complete.csv')

In [10]:
from model_components import preprocess_sentence, get_angles, positional_encoding, create_padding_mask, create_look_ahead_mask, FullyConnected, EncoderLayer, Encoder, DecoderLayer, Decoder, Transformer, CustomSchedule

In [5]:
df_en_de = pd.read_csv('df_complete.csv')

In [8]:
# pre-process sentences using helper function

pairs = df_en_de
pairs = pairs.sample(frac = 0.001)

In [9]:
len(pairs)

2172

In [10]:
# define source and target

source = pairs['german']
target = pairs ['english']

In [11]:
# create tokenizer & tensor for source and target
source_sentence_tokenizer= Tokenizer(filters='')
source_sentence_tokenizer.fit_on_texts(source)
source_tensor = source_sentence_tokenizer.texts_to_sequences(source)
source_tensor= tf.keras.preprocessing.sequence.pad_sequences(source_tensor, padding='post' )

target_sentence_tokenizer= Tokenizer(filters='')
target_sentence_tokenizer.fit_on_texts(target)
target_tensor = target_sentence_tokenizer.texts_to_sequences(target)
target_tensor= tf.keras.preprocessing.sequence.pad_sequences(target_tensor, padding='post' )

In [12]:
# Create word to index and index to word mappings for source and target

source_word_index = source_sentence_tokenizer.word_index
target_word_index = target_sentence_tokenizer.word_index

source_index_word = source_sentence_tokenizer.index_word
target_index_word = target_sentence_tokenizer.index_word

<h3> Convert Dictionary into DataFrame + Convert DataFrame into Dictionary </h3>

In [13]:
# Convert dictionary into dataframe
# This will be used in the training environment. 
# Once you fit the tokenizer to the training set and create a word_index dictionary,
# you save the dictionary as a csv file. 
def df_word_index(dictionary):
    df = pd.DataFrame.from_dict(dictionary, orient = 'index', columns= ['index']).reset_index()
    df = df.rename(columns = {'level_0':'word'})
    return df

def df_to_dict (df):
    dict_word_index = {row['word']:row['index'] for index, row in df.iterrows()}
    dict_index_word = {row['index']: row['word'] for index, row in df.iterrows()}
    return dict_word_index, dict_index_word

In [14]:
# convert dictionary into dataframes
df_source_word_index = df_word_index(source_word_index)
df_target_word_index = df_word_index(target_word_index)

# save dataframes as csv files; they will be loaded in the "translate_v1.ipynb" script
df_source_word_index.to_csv('df_source_word_index.csv', index = False)
df_target_word_index.to_csv('df_target_word_index.csv', index = False)

In [15]:
vocab_len_source = len(source_word_index.keys())
vocab_len_target = len(target_word_index.keys())
vocab_len_source, vocab_len_target

(8715, 6064)

In [16]:
num_tokens_source = vocab_len_source + 1
num_tokens_target = vocab_len_target + 1

In [17]:
source_train_tensor, source_test_tensor, target_train_tensor, target_test_tensor = train_test_split(
                                                                source_tensor, target_tensor,test_size=0.2
                                                                )

In [18]:
# save numpy array as csv file:
np.savetxt('source_train_tensor.csv', source_train_tensor, delimiter = ',')
np.savetxt('source_test_tensor.csv', source_test_tensor, delimiter = ',')
np.savetxt('target_train_tensor.csv', target_train_tensor, delimiter = ',')
np.savetxt('target_test_tensor.csv', target_test_tensor, delimiter = ',')


"""
For translating mode
"""
# # load numpy array from csv file:
# load_ar = np.loadtxt('source_train_tensor.csv', delimiter = ',')

'\nFor translating mode\n'

In [19]:
max_source_length= max(len(t) for t in source_tensor)
max_target_length= max(len(t) for t in  target_tensor)


In [20]:
max_source_length, max_target_length

(107, 110)

In [21]:
BATCH_SIZE = 32
#Create training dataset and shuffle
dataset_train = tf.data.Dataset.from_tensor_slices((source_train_tensor, target_train_tensor)).shuffle(BATCH_SIZE)
# divide into batches
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)

#Create test dataset
dataset_test = tf.data.Dataset.from_tensor_slices((source_test_tensor, target_test_tensor)).shuffle(BATCH_SIZE)
dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)


2022-08-18 12:24:26.293468: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
source_batch_train, target_batch_train =next(iter(dataset_train))
print(source_batch_train.shape, target_batch_train.shape)



(32, 107) (32, 110)


<h3> Parameters 6 </h3>

In [23]:
# Transformer arguments: 
# num_layers, embedding_dim, num_heads, fully_connected_dim, input_vocab_size, 
# target_vocab_size, max_positional_encoding_input,
# max_positional_encoding_target, dropout_rate=0.1, layernorm_eps=1e-6

num_layers = 4
embedding_dim = 64
num_heads = 5
fully_connected_dim = 512
input_vocab_size = num_tokens_source
target_vocab_size = num_tokens_target
max_positional_encoding_input = max_source_length
max_positional_encoding_target = max_target_length

<h3> Modify parameter number </h3>

In [24]:
# params_3 = f'\n params_3: num_layers: {num_layers}, embedding_dim: {embedding_dim}, num_heads: {num_heads}, \
# fully_connected_dim: {fully_connected_dim}, input_vocab_size: {input_vocab_size}, target_vocab_size: {target_vocab_size}, \
# max_positional_encoding_input: {max_positional_encoding_input}, max_positional_encoding_target: {max_positional_encoding_target}'

In [25]:
# Transformer arguments: 
# num_layers, embedding_dim, num_heads, fully_connected_dim, input_vocab_size, 
# target_vocab_size, max_positional_encoding_input,
# max_positional_encoding_target, dropout_rate=0.1, layernorm_eps=1e-6

transformer = Transformer(
    num_layers=num_layers,
    embedding_dim=embedding_dim,
    num_heads=num_heads,
    fully_connected_dim=fully_connected_dim,
    input_vocab_size=input_vocab_size,
    target_vocab_size=target_vocab_size,
    max_positional_encoding_input = max_positional_encoding_input,
    max_positional_encoding_target = max_positional_encoding_target
    )

(1, 107, 64)
(1, 110, 64)


- Create optimizer
- Use customised learning rate as defined in 'Attention Is All You Need' paper
- The learning rate increases linearly until "warmup_steps" training steps, then decays asymptotically
- Inputs: d_model, warmup_steps (default = 4000)

In [26]:
learning_rate = CustomSchedule(embedding_dim, warmup_steps = 2500)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [27]:
# define loss object
# from_logits = False, because we apply softmax to final Dense layer of Transformer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction='none')


In [28]:
def loss_function(real, pred):
                                                            # real = (m, Ty)
                                                            # pred = (m, Ty, num_tokens_target)

  mask = tf.math.logical_not(tf.math.equal(real, 0))        # want to select only non-zero values
                                                            # mask = (m, Ty), and is "True" for non-zero values

  loss_ = loss_object(real, pred)                           # compute loss for each time-step
                                                            # loss = (m, Ty)

  mask = tf.cast(mask, dtype=loss_.dtype)                   
  loss_ *= mask                                             # only count loss from non-zero values

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)           # divide sum(loss) by number of non-zero values

def accuracy_function(real, pred):                          # pred = (m, Ty, num_tokens_target)
  
  accuracies = tf.equal(real, tf.cast(tf.argmax(pred, axis=2), tf.int32))      # accuracies = (m, Ty) -- binary values

  mask = tf.math.logical_not(tf.math.equal(real, 0))        # mask = (m, Ty) -- boolean values
  accuracies = tf.math.logical_and(mask, accuracies)        # suppress values where real value is 0

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)      # divide sum of 1s in "accuracies" by sum of 1s in "mask"

In [29]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.Mean(name = 'test_accuracy')

In [30]:
@tf.function
def train_step(inp, tar):
                            # inp = (m, Tx)
                            # tar = (m, Ty)


  tar_inp = tar[:, :-1]     # "start_" to last word
  tar_real = tar[:, 1:]     # first word to "_end"

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inputs = (inp, tar_inp),
                                 training = True)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  acc = accuracy_function(tar_real, predictions)

  # store cumulative loss and acc in train_loss and train_accuracy
  train_loss(loss)
  train_accuracy(acc)

In [31]:
def compute_test_loss_acc(inp, tar):
    # inp = (m, Tx)
    # tar = (m, Ty)

    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    predictions, _ = transformer (inputs = (inp, tar_inp), training = False)
    test_loss = loss_function(tar_real, predictions)
    test_acc = accuracy_function(tar_real, predictions)

    return test_loss, test_acc

In [32]:
# checkpoint_path = './checkpoints/train'

# ckpt = tf.train.Checkpoint(optimizer=optimizer,
#                                  transformer=transformer)

# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 3)
# if ckpt_manager.latest_checkpoint:
#     ckpt.restore(ckpt_manager.latest_checkpoint)
#     print('Latest checkpoint restored!')

In [ ]:
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

In [35]:
EPOCHS = 10
# train_loss_dict = {}
# train_acc_dict = {}
# test_loss_dict = {}
# test_acc_dict = {}

for epoch in range(EPOCHS):
  start = time.time()

  # reset tf Mean objects
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()

  # iterate over every batch (= (inp, tar) tuple) in training dataset
  for (batch, (inp, tar)) in enumerate(dataset_train):
    train_step(inp, tar)

    if batch % 50 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} -- Train_Loss: {train_loss.result():.4f} Train_Accuracy: {train_accuracy.result():.4f}')

  #if (epoch+1) % 1 == 0:
    #ckpt_save_path = ckpt_manager.save()
    #print(f'Saving checkpoint for epoch {epoch + 1} at {ckpt_save_path}')
  
  # after one epoch of training, compute test loss and test acc
  for (batch, (inp, tar)) in enumerate(dataset_test):
    test_loss_batch, test_accuracy_batch = compute_test_loss_acc(inp, tar)
    # Update tf Mean objects
    test_loss(test_loss_batch)
    test_accuracy(test_accuracy_batch)
  


  print(f'Summary -- Epoch {epoch + 1} Train_Loss: {train_loss.result():.4f} Train_Accuracy: {train_accuracy.result():.4f} \
    Test_Loss: {test_loss.result():.4f} Test_Accuracy: {test_accuracy.result():.4f}')
  
  train_loss_list.append (train_loss.result().numpy())
  train_acc_list.append(train_accuracy.result().numpy())

  test_loss_list.append(test_loss.result().numpy())
  test_acc_list.append(test_accuracy.result().numpy())

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Epoch 1 Batch 0 -- Train_Loss: 4.1068 Train_Accuracy: 0.2891
Epoch 1 Batch 50 -- Train_Loss: 4.0722 Train_Accuracy: 0.3123
Summary -- Epoch 1 Train_Loss: 4.0722 Train_Accuracy: 0.3131     Test_Loss: 6.0572 Test_Accuracy: 0.1463
Time taken for 1 epoch: 59.12 secs

Epoch 2 Batch 0 -- Train_Loss: 4.0118 Train_Accuracy: 0.3277
Epoch 2 Batch 50 -- Train_Loss: 3.9154 Train_Accuracy: 0.3280
Summary -- Epoch 2 Train_Loss: 3.9141 Train_Accuracy: 0.3284     Test_Loss: 6.1273 Test_Accuracy: 0.1488
Time taken for 1 epoch: 58.49 secs

Epoch 3 Batch 0 -- Train_Loss: 3.8215 Train_Accuracy: 0.3148
Epoch 3 Batch 50 -- Train_Loss: 3.7642 Train_Accuracy: 0.3408
Summary -- Epoch 3 Train_Loss: 3.7646 Train_Accuracy: 0.3413     Test_Loss: 6.2507 Test_Accuracy: 0.1508
Time taken for 1 epoch: 58.51 secs

Epoch 4 Batch 0 -- Train_Loss: 3.7601 Train_Accuracy: 0.3338
Epoch 4 Batch 50 -- Train_Loss: 3.6303 Train_Accuracy: 0.3512
Summary -- Epoch 4 Train_Loss: 3.6307 Train_Accuracy: 0.3515     Test_Loss: 6.2399 Te

In [36]:
all_metrics = zip(train_loss_list, train_acc_list, test_loss_list, test_acc_list)
df_metrics = pd.DataFrame(all_metrics, columns = ['train_loss', 'train_acc', 'test_loss', 'test_acc']).reset_index().rename(columns = {'index':'epoch'})
df_metrics['epoch'] = df_metrics['epoch'].apply(lambda x: x+1)
df_metrics = df_metrics.apply(lambda x: round(x, 3))
df_metrics

,epoch,train_loss,train_acc,test_loss,test_acc
0,1,8.662,0.006,8.556,0.051
1,2,8.441,0.053,8.304,0.062
2,3,8.103,0.071,7.915,0.068
3,4,7.635,0.082,7.430,0.071
4,5,7.109,0.083,6.955,0.072
5,6,6.650,0.089,6.617,0.077
6,7,6.374,0.100,6.444,0.081
7,8,6.200,0.115,6.309,0.086
8,9,6.004,0.138,6.173,0.092
9,10,5.791,0.159,6.071,0.098


In [37]:
df_metrics.to_csv('df_metrics.csv', index = False)

In [38]:
# with open("params.txt", "a") as text_file:
#     text_file.write(params_3)
#     #text_file.write('params_3 -- time taken for 1 epoch: 54 secs')

In [39]:
file_path = 'saved_models/model'
transformer.save_weights(file_path,save_format='tf')

# # Recreate the exact same model purely from the file
# new_model = keras.models.load_model('path_to_my_model')